# Predicting House Sale Prices

We'll work with housing data for the city of Ames, Iowa, United States from 2006 to 2010. 
You can read more about why the data was collected here: http://ww2.amstat.org/publications/jse/v19n3/decock.pdf

You can also read about the different columns in the data:
https://ww2.amstat.org/publications/jse/v19n3/decock/DataDocumentation.txt

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

from sklearn.model_selection import KFold

In [3]:
df = pd.read_csv("AmesHousing.tsv", delimiter="\t")

In [4]:
df.head()

,Order,PID,MS SubClass,MS Zoning,Lot Frontage,Lot Area,Street,Alley,Lot Shape,Land Contour,...,Pool Area,Pool QC,Fence,Misc Feature,Misc Val,Mo Sold,Yr Sold,Sale Type,Sale Condition,SalePrice
0,1,526301100,20,RL,141.0,31770,Pave,NaN,IR1,Lvl,...,0,NaN,NaN,NaN,0,5,2010,WD,Normal,215000
1,2,526350040,20,RH,80.0,11622,Pave,NaN,Reg,Lvl,...,0,NaN,MnPrv,NaN,0,6,2010,WD,Normal,105000
2,3,526351010,20,RL,81.0,14267,Pave,NaN,IR1,Lvl,...,0,NaN,NaN,Gar2,12500,6,2010,WD,Normal,172000
3,4,526353030,20,RL,93.0,11160,Pave,NaN,Reg,Lvl,...,0,NaN,NaN,NaN,0,4,2010,WD,Normal,244000
4,5,527105010,60,RL,74.0,13830,Pave,NaN,IR1,Lvl,...,0,NaN,MnPrv,NaN,0,3,2010,WD,Normal,189900


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2930 entries, 0 to 2929
Data columns (total 82 columns):
Order              2930 non-null int64
PID                2930 non-null int64
MS SubClass        2930 non-null int64
MS Zoning          2930 non-null object
Lot Frontage       2440 non-null float64
Lot Area           2930 non-null int64
Street             2930 non-null object
Alley              198 non-null object
Lot Shape          2930 non-null object
Land Contour       2930 non-null object
Utilities          2930 non-null object
Lot Config         2930 non-null object
Land Slope         2930 non-null object
Neighborhood       2930 non-null object
Condition 1        2930 non-null object
Condition 2        2930 non-null object
Bldg Type          2930 non-null object
House Style        2930 non-null object
Overall Qual       2930 non-null int64
Overall Cond       2930 non-null int64
Year Built         2930 non-null int64
Year Remod/Add     2930 non-null int64
Roof Style         29

In [6]:
df.describe()

,Order,PID,MS SubClass,Lot Frontage,Lot Area,Overall Qual,Overall Cond,Year Built,Year Remod/Add,Mas Vnr Area,...,Wood Deck SF,Open Porch SF,Enclosed Porch,3Ssn Porch,Screen Porch,Pool Area,Misc Val,Mo Sold,Yr Sold,SalePrice
count,2930.00000,2.930000e+03,2930.000000,2440.000000,2930.000000,2930.000000,2930.000000,2930.000000,2930.000000,2907.000000,...,2930.000000,2930.000000,2930.000000,2930.000000,2930.000000,2930.000000,2930.000000,2930.000000,2930.000000,2930.000000
mean,1465.50000,7.144645e+08,57.387372,69.224590,10147.921843,6.094881,5.563140,1971.356314,1984.266553,101.896801,...,93.751877,47.533447,23.011604,2.592491,16.002048,2.243345,50.635154,6.216041,2007.790444,180796.060068
std,845.96247,1.887308e+08,42.638025,23.365335,7880.017759,1.411026,1.111537,30.245361,20.860286,179.112611,...,126.361562,67.483400,64.139059,25.141331,56.087370,35.597181,566.344288,2.714492,1.316613,79886.692357
min,1.00000,5.263011e+08,20.000000,21.000000,1300.000000,1.000000,1.000000,1872.000000,1950.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,2006.000000,12789.000000
25%,733.25000,5.284770e+08,20.000000,58.000000,7440.250000,5.000000,5.000000,1954.000000,1965.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,4.000000,2007.000000,129500.000000
50%,1465.50000,5.354536e+08,50.000000,68.000000,9436.500000,6.000000,5.000000,1973.000000,1993.000000,0.000000,...,0.000000,27.000000,0.000000,0.000000,0.000000,0.000000,0.000000,6.000000,2008.000000,160000.000000
75%,2197.75000,9.071811e+08,70.000000,80.000000,11555.250000,7.000000,6.000000,2001.000000,2004.000000,164.000000,...,168.000000,70.000000,0.000000,0.000000,0.000000,0.000000,0.000000,8.000000,2009.000000,213500.000000
max,2930.00000,1.007100e+09,190.000000,313.000000,215245.000000,10.000000,9.000000,2010.000000,2010.000000,1600.000000,...,1424.000000,742.000000,1012.000000,508.000000,576.000000,800.000000,17000.000000,12.000000,2010.000000,755000.000000


In [7]:
def transform_features(df):
    return df

def selection_features(df):
    return df[["Gr Liv Area", "SalePrice"]]

def train_and_test(data):
    train = data[:1460]
    test = data[1460:]

    # Select only numerical data types
    numerical_train = train.select_dtypes(include=['float', 'integer'])
    numerical_test = test.select_dtypes(include=['float', 'integer'])
    
    # Drop the target we want to predict
    features = numerical_train.columns.drop("SalePrice")
    
    lr = LinearRegression()
    lr.fit(numerical_train[features], numerical_train["SalePrice"])
    predictions = lr.predict(numerical_test[features])
    mse = mean_squared_error(predictions, numerical_test["SalePrice"])
    rmse = np.sqrt(mse)
    return rmse

trans_df = transform_features(df)
select_df = selection_features(trans_df)
rmse = train_and_test(select_df)
print(rmse)

57088.25161263909


# Feature Engineering
Handle missing values:
All columns:
Drop any with 5% or more missing values for now.

Text columns:
Drop any with 1 or more missing values for now.

Numerical columns:
For columns with missing values, fill in with the most common value in that column

1: All columns: Drop any with 5% or more missing values for now.

In [8]:
df_has_null = df.isnull().sum()
# Filter out missing data rate > 5%
drop_cols_series = df_has_null[df_has_null > len(df)*0.05].sort_values()
drop_cols_series

Garage Type       157
Garage Yr Blt     159
Garage Finish     159
Garage Qual       159
Garage Cond       159
Lot Frontage      490
Fireplace Qu     1422
Fence            2358
Alley            2732
Misc Feature     2824
Pool QC          2917
dtype: int64

In [9]:
df = df.drop(drop_cols_series.index, axis=1)

2: Text columns: Drop any with 1 or more missing values for now.

In [10]:
text_col_mv = df.select_dtypes(include=['object']).isnull().sum().sort_values(ascending=False)

## Filter Series to columns containing *any* missing values
drop_text_col_series = text_col_mv[text_col_mv > 0]

df = df.drop(drop_text_col_series.index, axis=1)

3: Numerical columns: For columns with missing values, fill in with the most common value in that column

In [11]:
num_col_mv = df.select_dtypes(include=['integer', 'float']).isnull().sum()
fixable_num_col_series = num_col_mv[(num_col_mv > 0) & (num_col_mv < len(df)*0.05)].sort_values()
fixable_num_col_series

BsmtFin SF 1       1
BsmtFin SF 2       1
Bsmt Unf SF        1
Total Bsmt SF      1
Garage Cars        1
Garage Area        1
Bsmt Full Bath     2
Bsmt Half Bath     2
Mas Vnr Area      23
dtype: int64

In [12]:
replace_dict = df[fixable_num_col_series.index].mode().to_dict(orient='records')[0]

In [13]:
## Use `pd.DataFrame.fillna()` to replace missing values.
df = df.fillna(replace_dict)

In [14]:
df.isnull().sum().value_counts()

0    64
dtype: int64

In [15]:
years_sold = df['Yr Sold'] - df['Year Built']
years_sold[years_sold < 0]

2180   -1
dtype: int64

In [16]:
years_since_remod = df['Yr Sold'] - df['Year Remod/Add']
years_since_remod[years_since_remod < 0]

1702   -1
2180   -2
2181   -1
dtype: int64

In [17]:
## Create new columns
df['Years Before Sale'] = years_sold
df['Years Since Remod'] = years_since_remod

## Drop rows with negative values for both of these new features
df = df.drop([1702, 2180, 2181], axis=0)

## No longer need original year columns
df = df.drop(["Year Built", "Year Remod/Add"], axis = 1)

In [18]:
## Drop columns that aren't useful for ML
df = df.drop(["PID", "Order"], axis=1)

## Drop columns that leak info about the final sale
df = df.drop(["Mo Sold", "Sale Condition", "Sale Type", "Yr Sold"], axis=1)

Let's update transform_features()

In [19]:
def transform_features(df):
    df_has_null = df.isnull().sum()
    drop_cols_series = df_has_null[df_has_null > len(df)*0.05].sort_values()
    df = df.drop(drop_cols_series.index, axis=1)

    text_col_mv = df.select_dtypes(include=['object']).isnull().sum().sort_values(ascending=False)
    drop_text_col_series = text_col_mv[text_col_mv > 0]
    df = df.drop(drop_text_col_series.index, axis=1)

    num_col_mv = df.select_dtypes(include=['integer', 'float']).isnull().sum()
    fixable_num_col_series = num_col_mv[(num_col_mv > 0) & (num_col_mv < len(df)*0.05)].sort_values()
    replace_dict = df[fixable_num_col_series.index].mode().to_dict(orient='records')[0]
    df = df.fillna(replace_dict)

    years_sold = df['Yr Sold'] - df['Year Built']
    years_since_remod = df['Yr Sold'] - df['Year Remod/Add']    
    df['Years Before Sale'] = years_sold
    df['Years Since Remod'] = years_since_remod
    df = df.drop([1702, 2180, 2181], axis=0)

    df = df.drop(["PID", "Order", "Mo Sold", "Sale Condition", "Sale Type", "Yr Sold", "Year Built", "Year Remod/Add"], axis=1)
    return df

def select_features(df):
    return df[["Gr Liv Area", "SalePrice"]]

def train_and_test(df):  
    train = df[:1460]
    test = df[1460:]
    
    ## Use `pd.DataFrame.select_dtypes()` to specify column types
    ## and return only those columns as a data frame.
    numeric_train = train.select_dtypes(include=['integer', 'float'])
    numeric_test = test.select_dtypes(include=['integer', 'float'])
    
    ## Use `pd.Series.drop()` to drop a value.
    features = numeric_train.columns.drop("SalePrice")
    lr = LinearRegression()
    lr.fit(train[features], train["SalePrice"])
    predictions = lr.predict(test[features])
    mse = mean_squared_error(test["SalePrice"], predictions)
    rmse = np.sqrt(mse)
    
    return rmse

df = pd.read_csv("AmesHousing.tsv", delimiter="\t")
transform_df = transform_features(df)
filtered_df = select_features(transform_df)
rmse = train_and_test(filtered_df)

rmse

55275.367312413066

# Feature Selection

In [20]:
numerical_df = transform_df.select_dtypes(include=['float', 'integer'])
numerical_df
numerical_df.corr()

,MS SubClass,Lot Area,Overall Qual,Overall Cond,Mas Vnr Area,BsmtFin SF 1,BsmtFin SF 2,Bsmt Unf SF,Total Bsmt SF,1st Flr SF,...,Wood Deck SF,Open Porch SF,Enclosed Porch,3Ssn Porch,Screen Porch,Pool Area,Misc Val,SalePrice,Years Before Sale,Years Since Remod
MS SubClass,1.000000,-0.204638,0.040285,-0.067498,0.005502,-0.058276,-0.070936,-0.129873,-0.219990,-0.250497,...,-0.016283,-0.013309,-0.022963,-0.037988,-0.050696,-0.003451,-0.024356,-0.085128,-0.037655,-0.044886
Lot Area,-0.204638,1.000000,0.090426,-0.033487,0.114982,0.180192,0.084454,0.020899,0.241353,0.321311,...,0.152849,0.089202,0.023004,0.016619,0.056135,0.094421,0.038013,0.267520,-0.020882,-0.019848
Overall Qual,0.040285,0.090426,1.000000,-0.093907,0.418972,0.278274,-0.040551,0.268760,0.545792,0.474888,...,0.253120,0.290553,-0.139907,0.018537,0.042412,0.030638,-0.027685,0.801206,-0.596358,-0.570231
Overall Cond,-0.067498,-0.033487,-0.093907,1.000000,-0.132071,-0.050143,0.040846,-0.136758,-0.174840,-0.157746,...,0.021046,-0.067152,0.071287,0.043804,0.043915,-0.016822,0.047066,-0.101540,0.369267,-0.046373
Mas Vnr Area,0.005502,0.114982,0.418972,-0.132071,1.000000,0.284305,-0.014139,0.087535,0.378545,0.376205,...,0.158743,0.123395,-0.109775,0.014689,0.068366,0.005143,-0.022884,0.506983,-0.306163,-0.190821
BsmtFin SF 1,-0.058276,0.180192,0.278274,-0.050143,0.284305,1.000000,-0.053656,-0.487829,0.522775,0.439662,...,0.216577,0.105248,-0.100343,0.051644,0.098336,0.085531,0.015748,0.439284,-0.278070,-0.148456
BsmtFin SF 2,-0.070936,0.084454,-0.040551,0.040846,-0.014139,-0.053656,1.000000,-0.238304,0.093368,0.088282,...,0.099212,-0.003918,0.032310,-0.023347,0.062897,0.044386,-0.002650,0.006127,0.027250,0.062075
Bsmt Unf SF,-0.129873,0.020899,0.268760,-0.136758,0.087535,-0.487829,-0.238304,1.000000,0.414595,0.296824,...,-0.040532,0.113352,0.006913,-0.005256,-0.047593,-0.031912,-0.026767,0.182751,-0.129199,-0.166312
Total Bsmt SF,-0.219990,0.241353,0.545792,-0.174840,0.378545,0.522775,0.093368,0.414595,1.000000,0.792227,...,0.222595,0.222762,-0.084576,0.039180,0.078435,0.073808,-0.011832,0.644012,-0.409209,-0.298629
1st Flr SF,-0.250497,0.321311,0.474888,-0.157746,0.376205,0.439662,0.088282,0.296824,0.792227,1.000000,...,0.218907,0.212943,-0.065170,0.045636,0.102147,0.124920,-0.011628,0.635185,-0.310902,-0.241824


In [21]:
abs_corr_coeff = numerical_df.corr()['SalePrice'].abs().sort_values()

In [22]:
## Let's only keep columns with a correlation coefficient of larger than 0.4
abs_corr_coeff[abs_corr_coeff > 0.4]

BsmtFin SF 1         0.439284
Fireplaces           0.474831
TotRms AbvGrd        0.498574
Mas Vnr Area         0.506983
Years Since Remod    0.534985
Full Bath            0.546118
Years Before Sale    0.558979
1st Flr SF           0.635185
Garage Area          0.641425
Total Bsmt SF        0.644012
Garage Cars          0.648361
Gr Liv Area          0.717596
Overall Qual         0.801206
SalePrice            1.000000
Name: SalePrice, dtype: float64

In [23]:
transform_df = transform_df.drop(abs_corr_coeff[abs_corr_coeff < 0.4].index, axis=1)

Which categorical columns should we keep?

In [24]:
## Create a list of column names from documentation that are *meant* to be categorical
nominal_features = ["PID", "MS SubClass", "MS Zoning", "Street", "Alley", "Land Contour", "Lot Config", "Neighborhood", 
                    "Condition 1", "Condition 2", "Bldg Type", "House Style", "Roof Style", "Roof Matl", "Exterior 1st", 
                    "Exterior 2nd", "Mas Vnr Type", "Foundation", "Heating", "Central Air", "Garage Type", 
                    "Misc Feature", "Sale Type", "Sale Condition"]

In [25]:
transform_cat_cols = []
for col in nominal_features:
    if col in transform_df.columns:
        transform_cat_cols.append(col)

uniqueness_counts = transform_df[transform_cat_cols].apply(lambda col: len(col.value_counts())).sort_values()
## Aribtrary cutoff of 10 unique values
drop_nonuniq_cols = uniqueness_counts[uniqueness_counts > 10].index
transform_df = transform_df.drop(drop_nonuniq_cols, axis=1)

In [26]:
## Select just the remaining text columns and convert to categorical
text_cols = transform_df.select_dtypes(include=['object'])
for col in text_cols:
    transform_df[col] = transform_df[col].astype('category')
    
## Create dummy columns and add back to the dataframe!
transform_df = pd.concat([
    transform_df, 
    pd.get_dummies(transform_df.select_dtypes(include=['category']))
], axis=1)

Update select_features()

# Note: below is our final model after lots of clean up, feature engineering and feature selections. Runs Linear Regression and K Fold cross validation to get final optimal predict value.

In [27]:
def transform_features(df):
    num_missing = df.isnull().sum()
    drop_missing_cols = num_missing[(num_missing > len(df)/20)].sort_values()
    df = df.drop(drop_missing_cols.index, axis=1)
    
    text_mv_counts = df.select_dtypes(include=['object']).isnull().sum().sort_values(ascending=False)
    drop_missing_cols_2 = text_mv_counts[text_mv_counts > 0]
    df = df.drop(drop_missing_cols_2.index, axis=1)
    
    num_missing = df.select_dtypes(include=['integer', 'float']).isnull().sum()
    fixable_numeric_cols = num_missing[(num_missing < len(df)/20) & (num_missing > 0)].sort_values()
    replacement_values_dict = df[fixable_numeric_cols.index].mode().to_dict(orient='records')[0]
    df = df.fillna(replacement_values_dict)
    
    years_sold = df['Yr Sold'] - df['Year Built']
    years_since_remod = df['Yr Sold'] - df['Year Remod/Add']
    df['Years Before Sale'] = years_sold
    df['Years Since Remod'] = years_since_remod
    df = df.drop([1702, 2180, 2181], axis=0)

    df = df.drop(["PID", "Order", "Mo Sold", "Sale Condition", "Sale Type", "Year Built", "Year Remod/Add", "Yr Sold"], axis=1)
    return df

def select_features(df, coeff_threshold=0.4, uniq_threshold=10):
    numerical_df = df.select_dtypes(include=['integer', 'float'])
    abs_corr_coeffs = numerical_df.corr()['SalePrice'].abs().sort_values()
    df = df.drop(abs_corr_coeffs[abs_corr_coeffs < coeff_threshold].index, axis=1)
    
    nominal_features = ["PID", "MS SubClass", "MS Zoning", "Street", "Alley", "Land Contour", "Lot Config", "Neighborhood", 
                    "Condition 1", "Condition 2", "Bldg Type", "House Style", "Roof Style", "Roof Matl", "Exterior 1st", 
                    "Exterior 2nd", "Mas Vnr Type", "Foundation", "Heating", "Central Air", "Garage Type", 
                    "Misc Feature", "Sale Type", "Sale Condition"]
    
    transform_cat_cols = []
    for col in nominal_features:
        if col in df.columns:
            transform_cat_cols.append(col)

    uniqueness_counts = df[transform_cat_cols].apply(lambda col: len(col.value_counts())).sort_values()
    drop_nonuniq_cols = uniqueness_counts[uniqueness_counts > 10].index
    df = df.drop(drop_nonuniq_cols, axis=1)
    
    text_cols = df.select_dtypes(include=['object'])
    for col in text_cols:
        df[col] = df[col].astype('category')
    df = pd.concat([df, pd.get_dummies(df.select_dtypes(include=['category']))], axis=1)
    
    return df

def train_and_test(df, k=0):
    numeric_df = df.select_dtypes(include=['integer', 'float'])
    features = numeric_df.columns.drop("SalePrice")
    lr = LinearRegression()
    
    if k == 0:
        train = df[:1460]
        test = df[1460:]

        lr.fit(train[features], train["SalePrice"])
        predictions = lr.predict(test[features])
        mse = mean_squared_error(test["SalePrice"], predictions)
        rmse = np.sqrt(mse)

        return rmse
    
    if k == 1:
        # Randomize *all* rows (frac=1) from `df` and return
        shuffled_df = df.sample(frac=1, )
        train = df[:1460]
        test = df[1460:]
        
        lr.fit(train[features], train["SalePrice"])
        predictions_one = lr.predict(test[features])        
        
        mse_one = mean_squared_error(test["SalePrice"], predictions_one)
        rmse_one = np.sqrt(mse_one)
        
        lr.fit(test[features], test["SalePrice"])
        predictions_two = lr.predict(train[features])        
       
        mse_two = mean_squared_error(train["SalePrice"], predictions_two)
        rmse_two = np.sqrt(mse_two)
        
        avg_rmse = np.mean([rmse_one, rmse_two])
        print(rmse_one)
        print(rmse_two)
        return avg_rmse
    else:
        kf = KFold(n_splits=k, shuffle=True)
        rmse_values = []
        for train_index, test_index, in kf.split(df):
            train = df.iloc[train_index]
            test = df.iloc[test_index]
            lr.fit(train[features], train["SalePrice"])
            predictions = lr.predict(test[features])
            mse = mean_squared_error(test["SalePrice"], predictions)
            rmse = np.sqrt(mse)
            rmse_values.append(rmse)
        print(rmse_values)
        avg_rmse = np.mean(rmse_values)
        return avg_rmse

df = pd.read_csv("AmesHousing.tsv", delimiter="\t")
transform_df = transform_features(df)
filtered_df = select_features(transform_df)
rmse = train_and_test(filtered_df, k=4)

rmse

[25768.913278482196, 27913.939807881914, 36341.72456922251, 27273.619133955697]


29324.54919738558

In [28]:
rmse

29324.54919738558